In [1]:
%config IPCompleter.greedy=True

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from threading import Thread
from queue import Queue
import multiprocessing

In [3]:
xcelLocation = "../xcels/"
!export xcelLocation="../xcels/"

In [4]:
!ls $xcelLocation | grep ".xlsx" > xlFiles
tmp = !cat xlFiles
names = [name[:-5] for name in tmp]

In [5]:
def cleaner():
    for name in names:
        if os.path.getsize(xcelLocation + name + '.xlsx') < 10000:
            os.remove(xcelLocation + name + '.xlsx')
cleaner()

In [6]:
def convert(xcelLocation, xlFileName, returnAllMode=False):
    xl = None
    try:
        if (not Path(xcelLocation + xlFileName + '.csv').is_file()) or returnAllMode:
            xl = pd.read_excel(xcelLocation + xlFileName + ".xlsx", index_col=0,
                               header=[0], skiprows=[0,1])
            xl.to_csv(xcelLocation + xlFileName + '.csv', encoding='utf-8', index=False)
    except:
        xl = str(xlFileName)
    finally:
        return xl

In [7]:
def convertThread(threadname, q, qDFs, qErrors):
    while not q.empty():
        fileNames = q.get()
        q.task_done()
        for name in fileNames:
            tmp = convert(xcelLocation=xcelLocation, xlFileName=name, returnAllMode=True)
            if isinstance(tmp, str):
                qErrors.put(tmp)
            else:
                qDFs.put((tmp.copy(), name))
    print(str(threadname) + " done")


In [8]:
allDFs = []
allDFNames = []
batchSize = 10
i = 0
numThread = 16
workers = []

pool = multiprocessing.Pool(processes=numThread)
m = multiprocessing.Manager()
queue = m.Queue()
qDFs = m.Queue()
qErrors = m.Queue()

while i*batchSize < len(names):
    if (i+1)*batchSize < len(names):
        queue.put(names[i*batchSize:(i+1)*batchSize])
    else:
        queue.put(names[i*10:])
    i+=1
print(len(names))
print(queue.qsize())

for i in range(numThread):
#     workers.append(Thread(target=readThread, args=("Thread-" + str(i), queue, qsum, qcount)))
#     workers.append(pool.apply_async(readThread, ("Thread-" + str(i), queue, qsum, qcount,)))
    workers.append(multiprocessing.Process(target=convertThread, args=("Thread-" + str(i),
                                                                    queue, qDFs, qErrors)))
    workers[i].start()

for i in range(numThread):
    workers[i].join()

while not qDFs.empty():
    dftmp, nametmp = qDFs.get()
    allDFs.append(dftmp)
    allDFNames.append(nametmp)

4410
441
Thread-7 done
Thread-2 done
Thread-3 done
Thread-9 done
Thread-12 done
Thread-13 done
Thread-15 done
Thread-11 done
Thread-8 done
Thread-1 done
Thread-6 done
Thread-10 done
Thread-4 done
Thread-0 done
Thread-5 done
Thread-14 done


In [9]:
print(len(allDFs))
def makeMasterTable():
    for index, df in enumerate(allDFs):
        year, month, day = allDFNames[index].split("-")
        yearlist = np.full(len(df), year).tolist()
        monthlist = np.full(len(df), month).tolist()
        daylist = np.full(len(df), day).tolist()
        df["year"] = yearlist
        df["month"] = monthlist
        df["day"] = daylist
    xl = pd.concat(allDFs, keys=allDFNames, ignore_index=True)
    xl.to_csv(xcelLocation + 'master.csv', encoding='utf-8', index=False)
    return xl
allDF = makeMasterTable()

4226


In [10]:
allDF.loc[allDF["نام"] == "شير و گوشت زاگرس شهركرد"]

,نام,تعداد,حجم,ارزش,دیروز,اولین,آخرین معامله - مقدار,آخرین معامله - تغییر,آخرین معامله - درصد,قیمت پایانی - مقدار,قیمت پایانی - تغییر,قیمت پایانی - درصد,کمترین,بیشترین,year,month,day
886944,شير و گوشت زاگرس شهركرد,6403,4949729,29629127883,5811.0,6101.0,5850.0,39.0,0.67,5986.0,175.0,3.01,5796.0,6406.0,1396,11,14
888622,شير و گوشت زاگرس شهركرد,1317,458178,1678764192,3490.0,3664.0,3664.0,174.0,4.99,3664.0,174.0,4.99,3664.0,3664.0,1396,10,27
889752,شير و گوشت زاگرس شهركرد,748,778168,4430460770,5676.0,5650.0,5555.0,-121.0,-2.13,5693.0,17.0,0.30,5540.0,5825.0,1396,11,28
894586,شير و گوشت زاگرس شهركرد,4910,2830354,16195399339,5986.0,5751.0,5687.0,-299.0,-4.99,5722.0,-264.0,-4.41,5687.0,5840.0,1396,11,15
896244,شير و گوشت زاگرس شهركرد,10176,4186066,21151553268,4913.0,4855.0,5158.0,245.0,4.99,5053.0,140.0,2.85,4855.0,5158.0,1396,11,8
897349,شير و گوشت زاگرس شهركرد,5862,3138878,17314643296,5722.0,5510.0,5603.0,-119.0,-2.08,5516.0,-206.0,-3.60,5436.0,5636.0,1396,11,16
898557,شير و گوشت زاگرس شهركرد,252,240758,1238879941,5155.0,5200.0,5090.0,-65.0,-1.26,5146.0,-9.0,-0.17,5080.0,5200.0,1396,12,20
899579,شير و گوشت زاگرس شهركرد,1209,890854,4851123352,5693.0,5555.0,5400.0,-293.0,-5.15,5445.0,-248.0,-4.36,5200.0,5557.0,1396,11,29
906146,شير و گوشت زاگرس شهركرد,3008,2367520,13692615317,5516.0,5600.0,5791.0,275.0,4.99,5784.0,268.0,4.86,5555.0,5791.0,1396,11,17
907331,شير و گوشت زاگرس شهركرد,362,305621,1536246329,5146.0,5110.0,5003.0,-143.0,-2.78,5027.0,-119.0,-2.31,5000.0,5110.0,1396,12,21
